# Artificial Neural Network

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.0'

Data Preprocessing

In [ ]:
data=pd.read_csv('Churn_Modelling.csv')

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
x=data.iloc[: , 3:-1].values
y=data.iloc[:,-1].values

Encoding categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))


In [ ]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


spiliting dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x,y, test_size=0.2)

scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

Building ANN

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 1 , activation='sigmoid'))

Training the ANN

In [ ]:
#COMPILE
ann.compile( optimizer = 'adam'  , loss = 'binary_crossentropy', metrics = ['accuracy'])
#train to training set
ann.fit(x_train , y_train, batch_size=32 , epochs= 100 )

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5696 - accuracy: 0.7220
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4673 - accuracy: 0.7997
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4422 - accuracy: 0.8005
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4273 - accuracy: 0.8039
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8105
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4027 - accuracy: 0.8159
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3904 - accuracy: 0.8270
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3797 - accuracy: 0.8409
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3710 - accuracy: 0.8453
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3641 - accura

Predict a piece manually

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


Predicting test set

In [ ]:
y_pred=ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)),1))

[[0 0]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

cm = confusion_matrix( y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1520   51]
 [ 234  195]]


0.8575